In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [18]:
hand_numbers =[np.load(f"hand_number/{n}.npy") for n in range(6)]

for i, n in enumerate(hand_numbers):
    print(i, n.shape)

0 (192, 21, 3)
1 (299, 21, 3)
2 (276, 21, 3)
3 (218, 21, 3)
4 (217, 21, 3)
5 (256, 21, 3)


In [19]:
def plot_landmarks(landmarks):
    landmarks = np.array([l - l.mean(axis=0) for l in landmarks])
    fig = plt.figure(figsize=(6, 4))
    gs = fig.add_gridspec(12, 1)
    ax = fig.add_subplot(gs[:-2, :], projection="3d")
    ax_slider = fig.add_subplot(gs[-1, :])
    slider = Slider(ax_slider, "index", 0, len(landmarks), valinit=0, valstep=1)

    scatters = [ax.scatter3D(*landmarks[idx, :, :].T) for idx in range(len(landmarks))]

    def update(v):
        for s in [s for s in scatters if s.get_visible()]:
            s.set_visible(False)
        idx = int(slider.val)
        scatters[idx].set_visible(True)

    slider.on_changed(update)
    update(None)

    ax.set(xlim=(-0.2, 0.2), ylim=(-0.4, 0.4), zlim=(0, 1), xlabel="x", ylabel="y", zlabel="z")
    ax.view_init(elev=95, azim=90)
    fig.tight_layout()


plot_landmarks(hand_numbers[4])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
hands = np.array([h - h.mean(axis=0) for h in np.concatenate(hand_numbers)])
labels = np.concatenate([np.array([i] * len(hn)) for i, hn in enumerate(hand_numbers)])
hands_train, hands_test, labels_train, labels_test = train_test_split(hands, labels)

In [34]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(21, 3)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(6, activation="softmax"),
    ]
)
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
model.fit(hands_train, labels_train, epochs=100, verbose=0)
model.evaluate(hands_test, labels_test)
# model.save("hand_number_tf")

12/12 [==============================] - 0s 673us/step - loss: 0.0018 - accuracy: 1.0000
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: hand_number_tf/assets


In [35]:
hands = np.array([h - h.mean(axis=0) for h in hand_numbers[4]])

model = tf.keras.models.load_model("hand_number_tf")
model.predict(hands[:5])

array([[9.8399778e-07, 1.4406377e-05, 1.9207293e-06, 3.9314933e-04,
        9.9820483e-01, 1.3847689e-03],
       [9.2800275e-07, 1.6252890e-05, 2.0983807e-06, 4.2220645e-04,
        9.9817061e-01, 1.3879142e-03],
       [8.8103172e-07, 1.4851270e-05, 1.8270048e-06, 4.1037909e-04,
        9.9828941e-01, 1.2826866e-03],
       [8.5274723e-07, 4.8068628e-06, 1.0395759e-06, 2.1585671e-04,
        9.9939108e-01, 3.8635448e-04],
       [1.4358204e-06, 6.6426583e-05, 3.7379873e-06, 9.5878867e-04,
        9.9247086e-01, 6.4987936e-03]], dtype=float32)